# NDEx Query Tutorial

In this tutorial you will learn to use the NDEx Client v2.0. The NDEx client is a module that simplifies access to the NDEx Server API and provides convenience methods for common operations on networks. 

This tutorial requires Python 3.6+ and the ndex2 module. 

See [the NDEx2 Client](https://github.com/ndexbio/ndex2-client) for installation instructions.

## Modules Required for this Tutorial

In [39]:
import ndex2.client as nc
import ndex2
import io
import json
from IPython.display import HTML
from time import sleep

## Setting up NDEx Clients

In this section you will configure two client objects to access the public NDEx server. 

The first will enable you to make anonymous requests. 

The second will enable you to perform operations requiring authentication, such as saving networks to your account.

### Anonymous Clients

The following code creates an NDEx client object to access the NDEx public server anonymously, then tests the client by getting the current server status.

In [40]:
anon_ndex=nc.Ndex2("http://public.ndexbio.org")
anon_ndex.update_status()
networks = anon_ndex.status.get("networkCount")
users = anon_ndex.status.get("userCount")
groups = anon_ndex.status.get("groupCount")
print("anon client: %s networks, %s users, %s groups" % (networks, users, groups))

anon client: 51799 networks, 1353 users, 124 groups


## Working with the NDEx Network Using the Anonymous Client

### Query STRING using a gene list

Find the reference network I want to use.

In [42]:
result_networks=anon_ndex.search_networks(search_string='STRING AND owner:"string"', size=10)
print("%s networks found" % (len(result_networks.get('networks'))))
print("\nNetworks:\n")
for ns in result_networks.get('networks'): print("  %s \t %s \t %s" % (ns.get('externalId'),ns.get('name'), ns.get('owner')) )

2 networks found

Networks:

  d14db454-3d18-11e8-a935-0ac135e8bacf 	 STRING-Human Protein Links 	 string
  275bd84e-3d18-11e8-a935-0ac135e8bacf 	 STRING - Human Protein Links - High Confidence (Score > 0.7) 	 string


### Query a Network – Neighborhood Query


You can retrieve a ‘neighborhood’ subnetwork of a network as a CX stream. The query finds the subnetwork by first 
identifying nodes that are associated with identifiers in the search_string, then traversing a specified number of 
edges starting from those nodes.  The **search_depth** parameter controls the search, defaults to 1 edge and can be no more than 3 edges. 

The query to anon_ndex will use the **get_neighborhood(network_id, search_string, search_depth=1, edge_limit=2500)** method to get a CX object. 


In [43]:
#!pip uninstall ndex2 -y
#!pip install ndex2 --no-cache-dir

Run neighborhood query on the [STRING - Human Protein Links - High Confidence (Score > 0.7)](http://public.ndexbio.org/#/network/275bd84e-3d18-11e8-a935-0ac135e8bacf) network

In [44]:
query_result_cx=anon_ndex.get_neighborhood('275bd84e-3d18-11e8-a935-0ac135e8bacf', 'XRN1',1, 6000)

for aspect in query_result_cx:
    if 'nodes' in aspect:
        number_of_nodes = len(aspect['nodes'])
    if 'edges' in aspect:
        number_of_edges = len(aspect['edges'])

print("Query result network contains %s nodes and %s edges." % (number_of_nodes, number_of_edges))


Query result network contains 56 nodes and 317 edges.


In [19]:
niceCX=ndex2.create_nice_cx_from_raw_cx(query_result_cx)
print ("Query result has %s aspects" % (len(query_result_cx))) 

Query result has 11 aspects


Use the niceCX data object in your analysis pipeline. For example,

In [45]:
niceCX.to_pandas_dataframe()

,source,interaction,target
0,YTHDC2,interacts with,XRN1
1,EXOSC5,interacts with,XRN1
2,SKIV2L2,interacts with,XRN1
3,ZFP36,interacts with,XRN1
4,NAA38,interacts with,XRN1
5,ERI1,interacts with,XRN1
6,LSM7,interacts with,XRN1
7,DDX60L,interacts with,XRN1
8,UPF1,interacts with,XRN1
9,SMG6,interacts with,XRN1


Display the search result as a network graph. See [Cytoscape-jupyter-widget](https://github.com/cytoscape/cytoscape-jupyter-widget/blob/develop/examples/WidgetDemo1.ipynb) for installation instructions.

Note: you don't have to convert the result to niceCX to render it.

In [46]:
from cyjupyter import Cytoscape
Cytoscape(data=query_result_cx, format='cx')

Cytoscape(data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': [{'name': '@context', …

Run other type of queries. For example, Interconnect query. The interconnect query is only availabe in NDEx2 client version 3.2.0 and above.

In [47]:
interconnectquery_result=anon_ndex.get_interconnectquery('275bd84e-3d18-11e8-a935-0ac135e8bacf', 
                                                         'XRN1 FOXO* M6PR ARF* BRCA* TP53',1, 6000)

for aspect in interconnectquery_result:
    if 'nodes' in aspect:
        number_of_nodes = len(aspect['nodes'])
    if 'edges' in aspect:
        number_of_edges = len(aspect['edges'])

print("Query result network contains %s nodes and %s edges." % (number_of_nodes, number_of_edges))



Query result network contains 22 nodes and 44 edges.


Display result:

In [48]:
from cyjupyter import Cytoscape
Cytoscape(data=interconnectquery_result, format='cx')

Cytoscape(data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': [{'name': '@context', …